# Currency Exchange Rates

## Example to show how to get all exchange rate for a currency

In [1]:
import numpy as np
import pandas as pd
import json
import os, sys

parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from src import file_check, yahoo as yh


### Load list of currencies
### Generate currencies exchange matrix

In [2]:
currency_file = parent_dir + '/data/currencies.csv'
forex_matrix_file = parent_dir + '/data/forex_matrix.csv'
forex_price_file = parent_dir + '/data/forex_price.json'

def getAllCurrenciesSymbols():
    df = pd.read_csv(currency_file, usecols = ['Currency code (ISO 4217)'])
    currency_list = df.values.flatten().tolist()
    remove_all = 'ALL'
    currency_list = [value for value in currency_list if value != remove_all]
    return currency_list

def createTwoDMmatrix(list_matrix):
    return [[i + j if i != j else 1 for i in list_matrix] for j in list_matrix]

if file_check.file_exist(forex_matrix_file):
    pass
else:  
    currency_list = getAllCurrenciesSymbols()
    forex_matrix = createTwoDMmatrix(getAllCurrenciesSymbols())
    df = pd.DataFrame(forex_matrix, columns = currency_list, index=currency_list)

    # Save to csv
    df.to_csv(forex_matrix_file)
    print('Matrix table created and saved')

### Get list of all currency exchange rates for a currency

In [3]:
def getAllExchangeVariableForCurrency(symbol):
    try:
        rdf = pd.read_csv(forex_matrix_file)
        if symbol in rdf:
            rdf = pd.read_csv(forex_matrix_file, usecols = [symbol])
            return rdf.values.flatten().tolist()
    except:
        print("Error get result for" + symbol)


# Save list of currency rates
forex_price_data = yh.getAllDetailExchangeRateFor(getAllExchangeVariableForCurrency('USD'))
json_string = json.dumps(forex_price_data, indent=2)
with open(forex_price_file, 'w') as outfile:
    outfile.write(json_string)

### Clean data and display table

In [4]:
df = pd.read_json(forex_price_file)
# df.info()

df = df[(df['symbol'] != "0") & (df['regularMarketPrice'] != "")]

df

,symbol,regularMarketPrice,regularMarketChange,regularMarketChangePercent,previous_close,open,bid,day_range,year_Week_range,ask
0,USDDZD,140.1360,-0.2220,(-0.1582%),140.3580,140.1360,140.1360,140.1360 - 140.1360,133.5986 - 146.8430,142.1360
1,USDAOA,422.4420,-0.1240,(-0.0294%),422.5663,422.4420,422.4420,422.4420 - 422.4420,422.4420 - 422.4420,434.7955
2,USDARS,137.7000,+0.3200,(+0.2329%),137.3800,137.7000,137.7000,137.7000 - 137.7000,96.9523 - 137.7000,137.7200
3,USDAMD,404.3200,-0.1200,(-0.0297%),404.4400,404.3200,404.3200,404.3200 - 404.3200,404.3200 - 404.3200,405.3200
4,USDAUD,1.4507,+0.0185,(+1.2938%),1.4322,1.4507,1.4507,1.4507 - 1.4510,1.3054 - 1.4963,1.4517
...,...,...,...,...,...,...,...,...,...,...
145,USDUYU,40.4310,-0.0570,(-0.1408%),40.4880,40.4880,40.4310,40.4310 - 40.4880,38.2517 - 44.3787,40.5310
146,USDUZS,"10,880.0000",-10.0000,(-0.0918%),"10,890.0000","10,883.3000","10,880.0000","10,880.0000 - 10,883.3000","10,569.4932 - 11,514.8105","10,920.0000"
147,USDVEF,"248,209.0000",0.0000,(0.0000%),"248,209.0000","248,209.0000","248,209.0000","248,209.0000 - 248,209.0000","248,209.0000 - 248,818.0000","248,259.0000"
148,USDVND,"23,412.0000",-11.0000,(-0.0470%),"23,423.0000","23,412.0000","23,412.0000","23,412.0000 - 23,412.0000","22,260.3535 - 23,480.0000","23,422.0000"
